In [40]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [ ]:
install_requirements()

In [42]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")
    load_dotenv()

    variables_to_check = ["OPENAI_API_KEY"]

    for var in variables_to_check:
        check_env(var)

In [43]:
setup_env()

OPENAI_API_KEY is set.


In [44]:
from openai import OpenAI
import os


class PromptOptimizer:
    """A simple prompt optimization class that uses OpenAI's Chat API to optimize prompts."""

    def __init__(self, prompt: str):
        """Initializes the PromptOptimizer with the given prompt. """
        self.ai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.prompt = prompt
        self.optimized_prompt = None

    def optimize(self, optimize_instructions = [
        "Refine the given prompt so that it is fit for use in production environments.",
        "Make the prompt concise, detailed and clear with a good balance of effectivenss and usefulness.",
    ], turns = 3) -> str:
        """Optimizes the prompt based on optimize instructions. """

        prompt_to_optimize = self.prompt
        previous_optimized_prompt = prompt_to_optimize
        for i in range(turns):
            try:
                print(f"Optimizing prompt - Turn {i+1}...")
                optimize_prompt = f"""
                    You will be given a prompt, refine it as per the given instructions. 
                    In addition to the instructions, come up with improvements and apply the refinements to the prompt.
                    Respond only with the final refined prompt after all improvements are made.
                    Instructions: ```{"\n".join(optimize_instructions)}```
                    Prompt: {previous_optimized_prompt}
                """
                system = "You are an AI prompt optimizer with deep understanding of LLM technology."
                ai_response = self.ai.chat.completions.create(
                    messages=[
                        {"role": "system", "content": system},
                        { "role": "user", "content": optimize_prompt }
                    ],
                    model="gpt-4o-mini"
                )
                previous_optimized_prompt = ai_response.choices[0].message.content
            except Exception as e:
                print(f"Failed to optimize prompt for turn {i+1}: {e}")
                continue
        self.optimized_prompt = previous_optimized_prompt
        return self.optimized_prompt
    
    def get_optimized_prompt(self) -> str:
        """Returns the optimized prompt."""
        if not self.optimized_prompt:
            optimized_prompt = self.optimize()
            if not optimized_prompt:
                print("Failed to optimize the prompt.")
        return self.optimized_prompt
    

    def evaluate(self):
        try:
            print("Evaluating the optimized prompt...")
            optimized_prompt = self.get_optimized_prompt()

            if not optimized_prompt:
                print("Failed to get the optimized prompt, can't evaluate.")
                return None
            
            evaluation_prompt = f"""
                    Given two prompts, compare the original prompt and the optimized prompt. 
                    Provide a table with the comparison of the two prompts along with reasons and scores.
                    In the markdown response provide a title, summary, and conclusion and the table. 
                    Respond in markdown format strictly. 
                    Make sure the markdown tables are compatible with Jupyter notebooks.
                    Original Prompt: {self.prompt}
                    Optimized Prompt: {optimized_prompt}
                """
            
            response = self.ai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are expert in prompt evaluation."},
                    {"role": "user", "content": evaluation_prompt}
                ])
            
            header = f"""# Prompt Optimization Report\n## Original Prompt\n{self.prompt}\n## Optimized Prompt\n{self.optimized_prompt}"""
            report = response.choices[0].message.content
            full_report = header + report
            return full_report
        except Exception as e:
            print(f"Failed to evaluate the optimized prompt: {e}")
            return None
    

        
        


In [45]:
from IPython.display import clear_output, Markdown

def run_prompt_optimizer(prompt: str) -> None:
    prompt_optimizer = PromptOptimizer(prompt)
    optimized_prompt = prompt_optimizer.get_optimized_prompt()
    clear_output()
    report = prompt_optimizer.evaluate()
    return Markdown(report)



In [ ]:
# Example 1
prompt = "Give me a detailed plan for learning Python Programming in 2025."

run_prompt_optimizer(prompt)


In [ ]:
# Example 2

prompt = "Give me the SOP of a conflict resolution strategy for a team of 5 members in a Big 4 Firm."


run_prompt_optimizer(prompt)